In [1]:
%load_ext autoreload
%autoreload 2

from dask_image.ndfilters import median_filter, threshold_local, percentile_filter, uniform_filter, gaussian_filter, sobel
from pyfires.PYF_WindowStats import get_mea_std_window, get_local_stats
from dask_image.ndfilters import convolve, maximum_filter
from satpy.modifiers.angles import _get_sun_angles
from pyfires.PYF_Anisotropy import aniso_diff
from scipy.ndimage import convolve as conv
from cython.parallel import prange
from tqdm import tqdm

import dask
dask.config.set(num_workers=8)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from datetime import datetime
from satpy import Scene
import dask.array as da
from glob import glob
from pyfires.PYF_detection import stage1_tests, run_basic_night_detection, run_dets

from pyfires.PYF_detection import stage1_tests, run_basic_night_detection, _make_kern
import pyfires.PYF_Consts as PYFc
from pyfires.PYF_basic import *

from pyspectral.rsr_reader import RelativeSpectralResponse
import xarray as xr
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from satpy import Scene
from glob import glob
import os

In [2]:
bdict = {'vi1_band': 'B03',
         'vi2_band': 'B06',
         'mir_band': 'B07',
         'lwi_band': 'B13'}


curfile = 'D:/sat_data/ahi_main/in/1650/HS_H09_20231029_1650_B07_FLDK_R20_S0110.DAT'
tod = 'D:/sat_data/ahi_main/out/'
pos = curfile.find('B07')
dtstr = curfile[pos-14:pos-1]
ifiles_l15 = glob(f'{os.path.dirname(curfile)}/*{dtstr}*.DAT')
scn = Scene(ifiles_l15, reader='ahi_hsd')
scn.load(['B03', 'B07', 'B13'])
scn = scn.resample(scn.coarsest_area(), resampler='native')
data_dict = initial_load(ifiles_l15,  # List of files to load
                         'ahi_hsd',   # The reader to use, in this case the AHI HSD reader
                         bdict)       # The bands to load
                         

cannot convert float NaN to integer


In [74]:
data_dict['PFP'] = stage1_tests(data_dict['MIR__BT'],
                                data_dict['BTD'],
                                data_dict['VI1_DIFF'],
                                data_dict['SZA'],
                                data_dict['LSM'],
                                ksizes=[5, 7, 9],
                                do_lsm_mask=True)

In [75]:
dt = data_dict['VI1_RAD'].data.compute()
print(np.nanmin(dt), np.nanmean(dt), np.nanmax(dt))

-0.0058026137 0.06181168 3.9206338


In [76]:
data_dict['VI1_DIFF_2'].data = da.where(np.isfinite(data_dict['VI1_DIFF_2']), data_dict['VI1_DIFF_2'], 0)
vid = np.array(data_dict['VI1_DIFF_2'])

In [77]:
vid

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [78]:
res_n1_n = aniso_diff(vid,
                niter=1,
                kappa=1)
res_n3_n = aniso_diff(vid,
                niter=2,
                kappa=1)
res_n5_n = aniso_diff(vid,
                niter=3,
                kappa=1)

main_n = np.dstack((vid, res_n1_n, res_n3_n, res_n5_n))

In [79]:
wrap_get_aniso_diffs = dask.delayed(get_aniso_diffs)
main_n2 = wrap_get_aniso_diffs(data_dict['VI1_DIFF_2'], [1, 2, 3])

In [80]:
main_std = np.nanstd(main_n, axis=2)

In [81]:
out = (main_std > 0.01).astype(np.uint8)
out = out * (vid > -0.15)

In [82]:
out2 = out * data_dict['PFP']
#save_output(scn, out2, f'out3', tod, 'B07' )

In [83]:
dets_arr = (out2 > 0).astype(np.uint8)
dets_arr[:,:] = 1
wrap_get_mean_std = dask.delayed(get_mea_std_window)
outa = wrap_get_mean_std(data_dict['PFP'].data,
                         data_dict['VI1_RAD'].data,  # VIS chan
                         data_dict['mi_ndfi'].data,  # NDFI
                         data_dict['LW1__BT'].data,  # LW Brightness Temperature
                         data_dict['BTD'].data,  # MIR-LW BTD
                         data_dict['MIR__BT'].data,  # MIR BT
                         data_dict['VI1_DIFF'].data,  # MIR-LWIR-VIS radiance diff
                         data_dict['LSM'].data,  # The land-sea mask
                         data_dict['LATS'].data,  # The pixel longitudes
                         255,  # The value denoting land in the LSM. If 255, ignore mask
                         25)

In [84]:
sdt = datetime.utcnow()
outan = outa.compute()
edt = datetime.utcnow()
print((edt - sdt).total_seconds())

117.521762


In [85]:
perc_good = outan[0, :, :]
n_winpix = outan[1, :, :]
n_cloudpix = outan[2, :, :]
n_waterpix = outan[3, :, :]
mean_lw = outan[4, :, :]
std_lw = outan[5, :, :]
mean_nd = outan[6, :, :]
std_nd = outan[7, :, :]
mean_vi = outan[8, :, :]
std_vi = outan[9, :, :]
mean_btd = outan[10, :, :]
std_btd = outan[11, :, :]
mean_mir = outan[12, :, :]
std_mir = outan[13, :, :]
mean_vid = outan[14, :, :]
std_vid = outan[15, :, :]

In [86]:
#save_output(scn, perc_good, f'perc_good', tod, 'B07' )
#save_output(scn, n_winpix, f'n_winpix', tod, 'B07' )
#save_output(scn, n_cloudpix, f'n_cloudpix', tod, 'B07' )
#save_output(scn, n_waterpix, f'n_waterpix', tod, 'B07' )
#save_output(scn, mean_lw, f'mean_lw', tod, 'B07' )
#save_output(scn, std_lw, f'std_lw', tod, 'B07' )
#save_output(scn, mean_nd, f'mean_nd', tod, 'B07' )
#save_output(scn, std_nd, f'std_nd', tod, 'B07' )
#save_output(scn, mean_vi, f'mean_vi', tod, 'B07' )
#save_output(scn, std_vi, f'std_vi', tod, 'B07' )
#save_output(scn, mean_btd, f'mean_btd', tod, 'B07' )
#save_output(scn, std_btd, f'std_btd', tod, 'B07' )
#save_output(scn, mean_mir, f'mean_mir', tod, 'B07' )
#save_output(scn, std_mir, f'std_mir', tod, 'B07' )
#save_output(scn, mean_vid, f'mean_vid', tod, 'B07' )
#save_output(scn, std_vid, f'std_vid', tod, 'B07' )

In [87]:
#save_output(scn, data_dict['VI1_DIFF'], f'VI1_DIFF', tod, 'B07' )
#save_output(scn, data_dict['MIR__BT'], f'MIR__BT', tod, 'B07' )
#save_output(scn, data_dict['LW1__BT'], f'LW1__BT', tod, 'B07' )
#save_output(scn, data_dict['mi_ndfi'], f'mi_ndfi', tod, 'B07' )
#save_output(scn, data_dict['BTD'], f'BTD', tod, 'B07' )
#save_output(scn, data_dict['VI1_RAD'], f'VI1_RAD', tod, 'B07' )

In [88]:
vi1_diff_stdm = (data_dict['VI1_DIFF'] - mean_vid) / std_vid
mir_bt_stdm = (data_dict['MIR__BT'] - mean_mir) / std_mir
lw1_bt_stdm = (data_dict['LW1__BT'] - mean_lw) / std_lw
mi_ndfi_stdm = (data_dict['mi_ndfi'] - mean_nd) / std_nd
btd_stdm = (data_dict['BTD'] - mean_btd) / std_btd
vi1_rad_stdm = (data_dict['VI1_RAD'] - mean_vi) / std_vi

In [89]:
out3 = out2 * (vi1_diff_stdm > vi1_rad_stdm * 1.5)
#save_output(scn, out3, f'out4', tod, 'B07' )

In [90]:
out4 = out3 * (mir_bt_stdm > 1.5)
#save_output(scn, out4, f'out5', tod, 'B07' )

In [91]:
vid2 = xr.where(mean_vid > 0, mean_vid, 0)
vi1_diff_std2m = (data_dict['VI1_DIFF'] - vid2) / std_vid
#save_output(scn, vi1_diff_std2m, f'VI1_DIFF_STD2', tod, 'B07' )

In [92]:
kern = np.ones((3,3))
fir_d_sum = conv(out4, kern)
local_max = maximum_filter(data_dict['VI1_DIFF'].data, (3,3))
out5 = (fir_d_sum == 1) * (data_dict['VI1_DIFF'] == local_max)
main_out = out4 * (fir_d_sum > 1) + out5 * out4

In [93]:
main_out = out4 * (fir_d_sum > 1) + out5 * out4
#save_output(scn, main_out, f'out6', tod, 'B07' )

In [94]:
main_out = main_out * xr.where(data_dict['MIR__BT'] > mean_mir + 2 * std_mir, main_out, 0)
#save_output(scn, main_out, f'out7', tod, 'B07' )

main_out = main_out * xr.where(data_dict['BTD'] > mean_btd + 2.5, main_out, 0)
#save_output(scn, main_out, f'out8', tod, 'B07' )

main_out = main_out * xr.where(data_dict['BTD'] > mean_btd + 2 * std_btd, main_out, 0)
#save_output(scn, main_out, f'out9', tod, 'B07' )

In [95]:
fir_d_sum = conv(main_out, kern)
local_max = maximum_filter(data_dict['MIR__BT'].data, (3,3))
tmp_out = (fir_d_sum == 1) * (data_dict['MIR__BT'] == local_max)
main_out = main_out * (fir_d_sum > 1) + main_out * tmp_out
#print(np.sum(main_out.data.compute()))
#save_output(scn, main_out, f'out10', tod, 'B07' )

In [96]:
kern = np.array([[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5],
                 [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]]) / 5

resx = np.abs(convolve(data_dict['MIR__BT'].data, kern))

kern = kern.T

resy = np.abs(convolve(data_dict['MIR__BT'].data, kern))

res = np.sqrt(resx*resx + resy*resy)

main_out = main_out * (res < 500)

In [97]:
#save_output(scn, resx, f'grad_x', tod, 'B07' )
#save_output(scn, resy, f'grad_y', tod, 'B07' )
#save_output(scn, res, f'grad_res', tod, 'B07' )

In [98]:
#save_output(scn, main_out, f'out11', tod, 'B07' )

In [99]:
mainer = np.array(main_out)
mirbt = np.array(data_dict['MIR__BT'])
viddr = np.array(data_dict['VI1_DIFF'])
btddr = np.array(data_dict['BTD'])

In [100]:
mirdif = np.zeros_like(mirbt)
viddif = np.zeros_like(viddr)
btddif = np.zeros_like(btddr)

for x in tqdm(range(1, viddr.shape[0]-1)):
    for y in range(1, viddr.shape[1]-1):
        
        if mainer[x, y] == 0: continue
            
        n_good = 0
            
        if mainer[x-1, y+1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x-1, y+1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x-1, y+1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x-1, y+1])
            n_good += 1
        if mainer[x, y+1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x, y+1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x, y+1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x, y+1])
            n_good += 1
        if mainer[x+1, y+1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x+1, y+1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x+1, y+1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x+1, y+1])
            n_good += 1
            
        if mainer[x-1, y] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x-1, y])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x-1, y])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x-1, y])
        if mainer[x+1, y] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x+1, y])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x+1, y])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x+1, y])
            n_good += 1
            
        if mainer[x-1, y-1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x-1, y-1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x-1, y-1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x-1, y-1])
            n_good += 1
        if mainer[x, y-1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x, y-1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x, y-1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x, y-1])
            n_good += 1
        if mainer[x+1, y-1] == 0: 
            mirdif[x, y] = mirdif[x, y] + (mirbt[x, y] - mirbt[x+1, y-1])
            viddif[x, y] = viddif[x, y] + (viddr[x, y] - viddr[x+1, y-1])
            btddif[x, y] = btddif[x, y] + (btddr[x, y] - btddr[x+1, y-1])
            n_good += 1
            
        # Account for pixels that may be in the centre of a group of PFPs and hence have no valid neighbours
        if n_good < 1:
            mirdif[x, y] = 1
            viddif[x, y] = 1
            btddif[x, y] = 1
        else:
            mirdif[x, y] = mirdif[x, y] / n_good
            viddif[x, y] = viddif[x, y] / n_good
            btddif[x, y] = btddif[x, y] / n_good
            

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5498/5498 [00:27<00:00, 197.58it/s]


In [101]:
locarr = get_local_stats(mainer, mirbt, btddr, viddr)
mirdif2 = locarr[:, :, 0]
btddif2 = locarr[:, :, 1]
viddif2 = locarr[:, :, 2]

In [102]:
main_out = main_out * (btddif > 1) * (viddif > 0.04)
main_out = main_out * (data_dict['BTD'] > mean_btd + std_mir + std_btd)
#save_output(scn, main_out, f'out14', tod, 'B07' )

In [103]:
kern = np.ones((3,3))
fir_d_sum = conv(main_out, kern)
local_max = maximum_filter(data_dict['VI1_DIFF'].data, (3,3))
out5 = (fir_d_sum == 1) * (data_dict['VI1_DIFF'] == local_max)
main_out2 = main_out * (fir_d_sum > 1) + out5 * main_out
#save_output(scn, main_out2, f'main_out2', tod, 'B07' )

In [104]:
kern_ones = np.ones((3,3))
fir_d_sum = conv(main_out2, kern_ones)
local_max = maximum_filter(data_dict['MIR__BT'].data, (3,3))
out5 = (fir_d_sum == 1) * (data_dict['MIR__BT'] == local_max)
main_out3 = main_out2 * (fir_d_sum > 1) + out5 * main_out2
#save_output(scn, main_out3, f'main_out3', tod, 'B07' )

In [105]:
#############
#TODOTODOTODO
# NEED TO ADD CODE TO RE-ADD MISSING 
# FIRE PIXELS IF THEY ARE BRIGHT ENOUGH 
# AND ARE NEXT TO EXISTING FIRE PIXELS
#TODOTODOTODO
#############

# Absolute MIR BT threshold before a pixel is declared 'fire'
mir_abs_thresh = 350
# BTD thresh for adding back missing pixels
min_btd_addback = 2
max_btd_addback = 15

main_out4 = main_out3 + xr.where(data_dict['MIR__BT'] > mir_abs_thresh, 1, 0).astype(np.uint8)
main_out4 = xr.where(main_out4 > 0, 1, 0).astype(np.uint8)

fir_d_sum = conv(main_out4, kern_ones)

# Threshold for adding missing fire pixels, as the algorithm removes some pixels adjacent to existing detections
# We add back using the BTD weighted by the number of fire pixels adjacent to the candidate.
btd_addback_thresh = (9 - fir_d_sum) * (8 / (max_btd_addback - min_btd_addback)) + min_btd_addback + mean_btd + std_btd
btd_addback_thresh = btd_addback_thresh * data_dict['PFP']
 
main_out5 = main_out4 + xr.where(data_dict['BTD'] > btd_addback_thresh, 1, 0).astype(np.uint8)
main_out5 = xr.where(main_out5 > 0, 1, 0).astype(np.uint8) * \
            xr.where(data_dict['PFP'] > 0, 1, 0).astype(np.uint8) * \
            xr.where(fir_d_sum > 0, 1, 0).astype(np.uint8)
#save_output(scn, main_out5, f'main_out5', tod, 'B07' )

In [106]:
data_dict['fire_dets'] = main_out3
data_dict['mean_mir'] = mean_mir
data_dict['mean_btd'] = mean_btd
data_dict['std_btd'] = std_btd

In [107]:
data_dict = calc_frp(data_dict)

In [108]:
save_output(scn, data_dict['frp_est'], f'frp_est', tod, 'B07' )

In [109]:
save_output(scn, data_dict['BTD'], f'BTD', tod, 'B07' )

In [3]:
scn = Scene(ifiles_l15, reader='ahi_hsd')
scn2 = Scene(ifiles_l15, reader='ahi_hsd')

scn.load(['B03', 'B07', 'B13'], calibration='radiance')
scn2.load(['B07', 'B13'])

scn = scn.resample(scn.coarsest_area(), resampler='native')
scn2 = scn2.resample(scn.coarsest_area(), resampler='native')
dd = sort_l1(scn['B03'],
             scn['B07'],
             scn['B13'],
             scn2['B07'],
             scn2['B13'],
             bdict
            )

cannot convert float NaN to integer


In [4]:
bob, bob2 = run_dets(dd)

In [ ]:
save_output(scn, bob, f'dets', tod, 'B07' )
save_output(scn, bob2, f'frp', tod, 'B07' )